![dssg_banner](assets/dssg_banner.png)

In [1]:
import chardet
import openai
import os
import pandas as pd
import numpy as np

from features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor import extract_knowledge_from_df
from dotenv import load_dotenv

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Agentic knowledge extractor

This notebook uses a Large Language Model (LLM) as an agent in order to extract knowledge from text input. The text input are the results from the fuzzy search, which returns the keyword found embedded in its context (words before and after the keyword). In fact, we are not interested in the keyword itself, but in its actual value. For example, the keyword may be "Firsthöhe" (= ridge height), so we'd like to find out the actual height. 

## Get API key

To be able to use the GPT API, add a file to the project root directory titled `.env` and add your API key to it: `OPENAI_API_KEY=[KEY]`

In [2]:
load_dotenv()
openai.api_key=os.getenv('OPENAI_API_KEY')

## Set variables

In [3]:
INPUT_FOLDER_PATH = "../data/nrw/bplan/features/keywords/fuzzy_search/traufhöhe.csv"
ID_COLUMN='filename'
TEXT_COLUMN='content'

Note that `nrows=10` to avoid accidentally spending large amounts. Remove the argument for a complete run.

In [4]:
with open(INPUT_FOLDER_PATH, 'rb') as f:
    enc = chardet.detect(f.read())

input_df = pd.read_csv(INPUT_FOLDER_PATH,
                       names=[ID_COLUMN, TEXT_COLUMN],
                       encoding = enc['encoding'],
                       header=None,
                       nrows=10)

## Specify keyword dictionary

Change the keyword _values_ (on the right hand side), if necessary.

In [5]:
keyword_dict = {
    'keyword':          'traufhöhe',
    'keyword_short':    'th',
    'template_name':    'features/textual_features/knowledge_extraction_agent/prompt_templates/knowledge_extraction_th.template',
    'filename':         'traufhöhe.csv'
    }

Note that an AuthenticationError is liklez caused bz a problem with your openAI API key.

In [6]:
all_responses = extract_knowledge_from_df(keyword_dict=keyword_dict,
                                          input_df=input_df,
                                          id_column_name=ID_COLUMN,
                                          text_column_name=TEXT_COLUMN)

2023-09-21 12:45:12.505 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:198 - Relevant keyword(s): traufhöhe
2023-09-21 12:45:12.507 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:199 - Extracting relevant info from text snippets via LLM agent.


2023-09-21 12:45:19.575 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:217 - Info extracted. Extracting numerical value from info.
2023-09-21 12:45:19.576 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:223 - Numerical values extracted. Validating their occurrence in input text.
2023-09-21 12:45:19.578 | INFO     | features.textual_features.knowledge_extraction_agent.agentic_knowledge_extractor:extract_knowledge_from_df:232 - Returning results for th.


In [7]:
all_responses.head(10)

,id,th_input,th_agent_response,th_extracted_value,validation
0,id,traufhöhe,None,NaN,True
1,1956227.pdf,"/ fh 9,5 m bei gebäuden mit zwei vollgeschossen: th 6,5 m / fh 8,5 m bestimmung der traufhöhe: die traufhöhe wird durch den äußeren schnittpunkt des aufgehenden mauerwerks mit der dachhaut gebildet. bestimmung der firsthöhe: die firsthöhe wird durch den ;;; grz 0,3 grundflächenzahl ii zahl der vollgeschosse als höchstmaß 2 wo höchstzulässige zahl der wohnungen in wohngebäuden th 00,0 m traufhöhe als höchstmaß fh 00,0 m firsthöhe als höchstmaß 00,00 m unterer bezugspunkt zur bemessung der trauf und firsthöhen bauweise, baulinien, ;;; 4,3 m / fh 9,5 m bei gebäuden mit zwei vollgeschossen: th 6,5 m / fh 8,5 m bestimmung der traufhöhe: die traufhöhe wird durch den äußeren schnittpunkt des aufgehenden mauerwerks mit der dachhaut gebildet. bestimmung der firsthöhe: die firsthöhe wird","th: 6,5 m",6.50,True
2,1956230.pdf,"anderes material zulässig. 2 dächer zulässig sind satteldächer mit einer neigung von 30° 40°. bei aneinandergebauten gebäuden sind dachneigung und traufhöhe einander anzupassen. dachaufbauten (gauben) dürfen 1/2 der gesamtdachlänge nicht überschreiten die traufe der gaube darf nicht höher als 1,20 m",None,NaN,True
3,2112722.pdf,"zulhsige grundflächenzahl (grz) (gern.§ 16 (2) nr.1 baunvo) offene bauweise (gern.§ 9 (1) nr. 2 baugb und§ 22 baunvo) ma,c. traufllöhe (gern.§ 9 (1) nr. 1 baugb u. § 16 (2) nr. 4 baunvo) m;1x. firsthöhe (gern.§ 9 (1) nr. 1",None,NaN,True
4,2112808.pdf,"der außenflächen der außenwand mit der dachhaut. untergeordnete bauteile (vorbauten, erker, zwerchgiebel) dürfen auf maximal 1/3 der baukörperlänge die maximale traufhöhe überschreiten. die maximal zulässige firsthöhe wird am fertiggestellten gebäude am schnittpunkt der außenflächen der dachhaut gemessen. maximal zulässige traufhöhe in ;;; maximale traufhöhe überschreiten. die maximal zulässige firsthöhe wird am fertiggestellten gebäude am schnittpunkt der außenflächen der dachhaut gemessen. maximal zulässige traufhöhe in metern th 5,00m auf flächen gem. § 9 (1) ziffer 25a baugb sind heimische sträucherund heister der qualität str.","th: 5,00 m",5.00,True
5,2220395_4.pdf,"der bebauungsplan den festsetzungen des ehemaligen bebauungsplanes nr. 5 und entspricht ebenso der derzeitigen bestandssituation. 7.4 höhe baulicher anlagen die traufhöhe für das eingeschossige gebäude wird mit maximal 4,75 m über der an grenzenden vorhandenen erschließungsstraße festgesetzt. damit wird gewährleistet, dass","th: 4,75 m",4.75,True
6,2220407_6.pdf,parallel zur öffentlichen verkehrsfläche auszubilden. form und höhe der vorhandenen attika zur öffentlichen verkehrsfläche sind beim satteldach zu übernehmen; die traufenhöhe wird auf 42.54 m ü. n.n. begrenzt. 3. auf den gebäudeschenkeln (anbauten) sind ausschließlich sattel und pultdächer zulässig. bei den ;;; und pultdächer zulässig. bei den pult und satteldächern sind form und höhe der vorhandenen attika entlang der grundstücksgrenzen einzuhalten. die traufenhöhe entlang der grundstücksgrenzen von anbauten wird auf 42.54 m ü. n.n. begrenzt. walm und krüppelwalmdächer sind ausgeschlossen. 4. dachüberstände sind,None,NaN,True
7,2220409_1.pdf,"41,5 m über nhn festgesetzt, was einem maß von rund 9,50 m über dem ge ländeniveau entspricht. die maximal zulässige traufhöhe beträgt 38,5 m über nhn, was rund 6,50 m über dem geländeniveau entspricht. es ist eine maximale zweigeschossige bebauung festgesetzt","th: 9,50 m\nth: 6,50 m\nNone",9.50,True
8,2220409_3.pdf,"passen sich an die umliegenden bestandsgebäude und ermöglichen eine flexib le anordnung und ausgestaltung neuer baukörper. zahl der vollgeschosse / traufhöhe (th) und firsthöhe (fh) für den gesamten geltungsbereich des vorhabenbezogenen bebauungsplans werden zwei vollgeschosse festgesetzt. die festsetzung leitet sich aus ;;; von 41,5 m über nhn festgesetzt, was einem maß von rund 9,50 m über dem geländeniveau entspricht. die maximal zulässige traufhöhe beträgt 3

## Reset if validation unsuccessful

If validation wasn't successful (= False), the extracted_value column should be set back to NaN.

In [8]:
column_name = f"{keyword_dict['keyword_short']}_extracted_value"
all_responses.loc[all_responses['validation'] == False, column_name] = np.nan

all_responses.head()

,id,th_input,th_agent_response,th_extracted_value,validation
0,id,traufhöhe,None,NaN,True
1,1956227.pdf,"/ fh 9,5 m bei gebäuden mit zwei vollgeschossen: th 6,5 m / fh 8,5 m bestimmung der traufhöhe: die traufhöhe wird durch den äußeren schnittpunkt des aufgehenden mauerwerks mit der dachhaut gebildet. bestimmung der firsthöhe: die firsthöhe wird durch den ;;; grz 0,3 grundflächenzahl ii zahl der vollgeschosse als höchstmaß 2 wo höchstzulässige zahl der wohnungen in wohngebäuden th 00,0 m traufhöhe als höchstmaß fh 00,0 m firsthöhe als höchstmaß 00,00 m unterer bezugspunkt zur bemessung der trauf und firsthöhen bauweise, baulinien, ;;; 4,3 m / fh 9,5 m bei gebäuden mit zwei vollgeschossen: th 6,5 m / fh 8,5 m bestimmung der traufhöhe: die traufhöhe wird durch den äußeren schnittpunkt des aufgehenden mauerwerks mit der dachhaut gebildet. bestimmung der firsthöhe: die firsthöhe wird","th: 6,5 m",6.5,True
2,1956230.pdf,"anderes material zulässig. 2 dächer zulässig sind satteldächer mit einer neigung von 30° 40°. bei aneinandergebauten gebäuden sind dachneigung und traufhöhe einander anzupassen. dachaufbauten (gauben) dürfen 1/2 der gesamtdachlänge nicht überschreiten die traufe der gaube darf nicht höher als 1,20 m",None,NaN,True
3,2112722.pdf,"zulhsige grundflächenzahl (grz) (gern.§ 16 (2) nr.1 baunvo) offene bauweise (gern.§ 9 (1) nr. 2 baugb und§ 22 baunvo) ma,c. traufllöhe (gern.§ 9 (1) nr. 1 baugb u. § 16 (2) nr. 4 baunvo) m;1x. firsthöhe (gern.§ 9 (1) nr. 1",None,NaN,True
4,2112808.pdf,"der außenflächen der außenwand mit der dachhaut. untergeordnete bauteile (vorbauten, erker, zwerchgiebel) dürfen auf maximal 1/3 der baukörperlänge die maximale traufhöhe überschreiten. die maximal zulässige firsthöhe wird am fertiggestellten gebäude am schnittpunkt der außenflächen der dachhaut gemessen. maximal zulässige traufhöhe in ;;; maximale traufhöhe überschreiten. die maximal zulässige firsthöhe wird am fertiggestellten gebäude am schnittpunkt der außenflächen der dachhaut gemessen. maximal zulässige traufhöhe in metern th 5,00m auf flächen gem. § 9 (1) ziffer 25a baugb sind heimische sträucherund heister der qualität str.","th: 5,00 m",5.0,True


In [10]:
all_responses.to_json('../data/nrw/bplan/knowledge_agent_output.json')